## Simulating the rest of the season
Using the team strenghts that we have created let's see if we can simualate the rest of the season

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn import linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 20)
from sklearn.pipeline import Pipeline
from os import listdir
from os.path import isfile, join
from scipy.stats import poisson


In [2]:
#had to remove a %inline matplotlib for this to work
from ipynb.fs.defs._04_PredictResultProb import home_vars_prep, away_vars_prep, Predicted_HG, Predicted_AG

In [3]:
data_folder = "C:/Users/Byron/Documents/Football Predictions/UnderstatPredictions/Datasets"
#results file
results_df = pd.read_csv(data_folder+"/UnderstatResultsAll.csv")
results_df= results_df[(results_df["league"]=="EPL")&
                      (results_df["season"]=="2019/20")
                      ]

In [4]:
#predictions file
pred_df = pd.read_csv(data_folder+"/xg_predictions.csv")
pred_df = pred_df[(pred_df["league"]=="EPL") &
                 (pred_df["season"]=="2019/20") &
                  (pred_df["AverageMethod"]=="EMA No reset")
                 ]
pred_df.tail()

,league,season,fixture,season_gw,Team_Home,Team_Away,homegoals,awaygoals,Result,AverageMethod,...,PredScoreProb,PredResult,PredResultProb,PredResultScore,PredResultScoreProb,FinalResultProb,FinalScore,PredCorrect,PredResultScoreCorrect,PredScoreCorrect
16444,EPL,2019/20,Leicester vs Manchester United,38,Leicester,Manchester United,NaN,NaN,NaN,EMA No reset,...,16.008885,A,36.383884,0 . 1,15.438770,36.383884,NaN,0,0,0
16448,EPL,2019/20,Everton vs Bournemouth,38,Everton,Bournemouth,NaN,NaN,NaN,EMA No reset,...,17.539342,H,55.578292,1 . 0,17.539342,17.361495,NaN,0,0,0
16452,EPL,2019/20,Crystal Palace vs Tottenham,38,Crystal Palace,Tottenham,NaN,NaN,NaN,EMA No reset,...,18.741890,A,36.776832,0 . 1,16.991094,36.776832,NaN,0,0,0
16456,EPL,2019/20,Chelsea vs Wolverhampton Wanderers,38,Chelsea,Wolverhampton Wanderers,NaN,NaN,NaN,EMA No reset,...,19.885326,H,40.776085,1 . 0,18.884967,24.639872,NaN,0,0,0
16460,EPL,2019/20,Burnley vs Brighton,38,Burnley,Brighton,NaN,NaN,NaN,EMA No reset,...,16.259228,H,38.686238,1 . 0,16.244876,28.806080,NaN,0,0,0


In [5]:
#team strength file
team_df = pd.read_csv(data_folder+'/UnderstatAttDefStr.csv')
team_df = team_df[(team_df["league"]=="EPL") &
                 (team_df["season"]=="2019/20") &
                  (team_df["AverageMethod"]=="EMA No reset")
                 ]
team_df.tail()

,league,season,matchid,matchdate,fixture,Team,Opponent,GoalsFor,GoalsAgainst,xG,...,xGA_av,GoalsFor_av,GoalsAgainst_av,AverageMethod,xG_av_gw,xGA_av_gw,GoalsFor_av_gw,GoalsAgainst_av_gw,AttStr,DefStr
8227,EPL,2019/20,11980,2020-04-18 14:00:00,Sheffield United vs Wolverhampton Wanderers,Wolverhampton Wanderers,Sheffield United,NaN,NaN,NaN,...,1.026419,1.382748,1.1282,EMA No reset,1.450846,1.447114,1.384968,1.378250,1.083274,0.709287
8228,EPL,2019/20,11992,2020-04-25 14:00:00,Wolverhampton Wanderers vs Everton,Wolverhampton Wanderers,Everton,NaN,NaN,NaN,...,1.026419,1.382748,1.1282,EMA No reset,1.448509,1.449066,1.387632,1.386383,1.085021,0.708331
8229,EPL,2019/20,11994,2020-05-02 14:00:00,Burnley vs Wolverhampton Wanderers,Wolverhampton Wanderers,Burnley,NaN,NaN,NaN,...,1.026419,1.382748,1.1282,EMA No reset,1.446542,1.451940,1.388690,1.394974,1.086497,0.706929
8230,EPL,2019/20,12012,2020-05-09 14:00:00,Wolverhampton Wanderers vs Crystal Palace,Wolverhampton Wanderers,Crystal Palace,NaN,NaN,NaN,...,1.026419,1.382748,1.1282,EMA No reset,1.441944,1.441229,1.376852,1.374644,1.089962,0.712183
8231,EPL,2019/20,12021,2020-05-17 14:00:00,Chelsea vs Wolverhampton Wanderers,Wolverhampton Wanderers,Chelsea,NaN,NaN,NaN,...,1.026419,1.382748,1.1282,EMA No reset,1.448217,1.447400,1.377846,1.375308,1.085240,0.709147


In [6]:
#find out what the latest gameweek data is for each team
latest_gw = team_df[team_df["GoalsFor"].isnull()].groupby(["league","season","Team"]).agg(latest_gw=("season_gw","min")).reset_index()

#take the team attack and defense strengths from each team by joining on to their last week
last_status = pd.merge(team_df,latest_gw,how = "inner",left_on =["league","season","Team","season_gw"]
                       ,right_on =["league","season","Team","latest_gw"])

#take the league average of Home Goals and Away Goals for indexing later
league_av = pd.merge(pred_df, latest_gw, how = "inner", left_on =["league","season","season_gw"]
                    , right_on =["league","season","latest_gw"])
league_av = league_av.groupby(["league","season"])["SeasonHG","SeasonAG"].mean().reset_index()
print(league_av)
#last_status = pd.merge(last_status,league_av, how = "inner",on=["league","season","season_gw"])
last_status = last_status[["league","season","season_gw","Team","AttStr","DefStr"]]
last_status.head(3)

  league   season  SeasonHG  SeasonAG
0    EPL  2019/20   1.49141  1.251026


,league,season,season_gw,Team,AttStr,DefStr
0,EPL,2019/20,28,Arsenal,1.021293,1.033239
1,EPL,2019/20,28,Aston Villa,0.727792,1.531214
2,EPL,2019/20,28,Bournemouth,0.867874,1.219172


In [7]:
#get the remaining fixtures (where goals scored is null)
rem_fixtures = results_df[results_df["homegoals"].isnull()]
rem_fixtures.head(11)

,league,season,matchid,matchdate,fixture,hometeam,awayteam,homegoals,awaygoals,hxG,axG,season_id,league_teams_count,season_gw,HomePoints,AwayPoints
4005,EPL,2019/20,11911,2020-02-24 20:00:00,Liverpool vs West Ham,Liverpool,West Ham,NaN,NaN,NaN,NaN,270,20,27,0,0
4006,EPL,2019/20,11919,2020-02-28 20:00:00,Norwich vs Leicester,Norwich,Leicester,NaN,NaN,NaN,NaN,271,20,28,0,0
4007,EPL,2019/20,11915,2020-02-29 12:30:00,Brighton vs Crystal Palace,Brighton,Crystal Palace,NaN,NaN,NaN,NaN,272,20,28,0,0
4008,EPL,2019/20,11913,2020-02-29 15:00:00,Aston Villa vs Sheffield United,Aston Villa,Sheffield United,NaN,NaN,NaN,NaN,273,20,28,0,0
4009,EPL,2019/20,11914,2020-02-29 15:00:00,Bournemouth vs Chelsea,Bournemouth,Chelsea,NaN,NaN,NaN,NaN,274,20,28,0,0
4010,EPL,2019/20,11916,2020-02-29 15:00:00,Everton vs Manchester United,Everton,Manchester United,NaN,NaN,NaN,NaN,275,20,28,0,0
4011,EPL,2019/20,11917,2020-02-29 15:00:00,Manchester City vs Arsenal,Manchester City,Arsenal,NaN,NaN,NaN,NaN,276,20,28,0,0
4012,EPL,2019/20,11918,2020-02-29 15:00:00,Newcastle United vs Burnley,Newcastle United,Burnley,NaN,NaN,NaN,NaN,277,20,28,0,0
4013,EPL,2019/20,11920,2020-02-29 15:00:00,Tottenham vs Wolverhampton Wanderers,Tottenham,Wolverhampton Wanderers,NaN,NaN,NaN,NaN,278,20,28,0,0
4014,EPL,2019/20,11922,2020-02-29 15:00:00,West Ham vs Southampton,West Ham,Southampton,NaN,NaN,NaN,NaN,279,20,28,0,0


In [8]:
#restructure home and away teams

join_list = ['league','season']
keep_list = ["Team","AttStr","DefStr"]

home_df = home_vars_prep(last_status,join_list,keep_list)
away_df = away_vars_prep(last_status,join_list,keep_list)
print(home_df.head(1))

single_line = rem_fixtures[["league","season","matchdate","fixture","season_gw","hometeam","awayteam"]].copy()
single_line = single_line.rename(columns={"hometeam":"Team_Home"
                                         , "awayteam":"Team_Away"})

single_line = pd.merge(single_line,home_df, on=['league', 'season','Team_Home'])
single_line = pd.merge(single_line,away_df, on=['league', 'season','Team_Away'])
single_line = pd.merge(single_line,league_av, on=['league', 'season'])
single_line.sort_values("matchdate",inplace = True)
single_line.head()

  league   season Team_Home  AttStr_Home  DefStr_Home
0    EPL  2019/20   Arsenal     1.021293     1.033239


,league,season,matchdate,fixture,season_gw,Team_Home,Team_Away,AttStr_Home,DefStr_Home,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG
0,EPL,2019/20,2020-02-24 20:00:00,Liverpool vs West Ham,27,Liverpool,West Ham,1.518461,0.614483,0.773659,1.358462,1.49141,1.251026
34,EPL,2019/20,2020-02-28 20:00:00,Norwich vs Leicester,28,Norwich,Leicester,0.720824,1.243807,1.086597,0.941559,1.49141,1.251026
12,EPL,2019/20,2020-02-29 12:30:00,Brighton vs Crystal Palace,28,Brighton,Crystal Palace,0.864810,1.026824,0.717497,1.026888,1.49141,1.251026
30,EPL,2019/20,2020-02-29 15:00:00,Bournemouth vs Chelsea,28,Bournemouth,Chelsea,0.867874,1.219172,1.242395,0.687826,1.49141,1.251026
43,EPL,2019/20,2020-02-29 15:00:00,West Ham vs Southampton,28,West Ham,Southampton,0.773659,1.358462,0.992140,0.975213,1.49141,1.251026


Apply Indexing to create predicted score

In [9]:
single_line["Pred_HG"] = single_line.apply(lambda row: Predicted_HG(
                                            row["AttStr_Home"],row["DefStr_Away"],row["SeasonHG"]), axis =1)
single_line["Pred_AG"] = single_line.apply(lambda row: Predicted_AG(
                                            row["DefStr_Home"],row["AttStr_Away"],row["SeasonAG"]), axis =1)

Poisson Distribution to create goals probability

In [10]:
#apply poisson distribution to predicted home and away goals. Depending on the number of fixtures you have this can take a minute.

single_line["Prob_HG"] = single_line["Pred_HG"].apply(lambda x: poisson(x).pmf(range(0,10)))
single_line["Prob_AG"] = single_line["Pred_AG"].apply(lambda x: poisson(x).pmf(range(0,10)))
single_line["score"] = single_line.apply(lambda row: pd.DataFrame(np.outer(row["Prob_HG"],row["Prob_AG"])*100), axis =1)

In [16]:
print(single_line.iloc[0,]["Pred_HG"])
print(single_line.iloc[0,]["Pred_AG"])
print(single_line.iloc[0,]["Prob_AG"])
print(single_line.iloc[0,]["Prob_HG"])
print(single_line.iloc[0,]["Prob_AG"])
print(single_line.iloc[0,]["score"])
print()


3.0764394615628796
0.5947371721787386
[5.51707551e-01 3.28120989e-01 9.75728745e-02 1.93434051e-02
 2.87606052e-03 3.42100020e-04 3.39099331e-05 2.88107110e-06
 2.14185010e-07 1.41537541e-08]
[0.04612319 0.1418952  0.21826599 0.22382737 0.17214784 0.10592048
 0.05430966 0.02386862 0.0091788  0.00313756]
[5.51707551e-01 3.28120989e-01 9.75728745e-02 1.93434051e-02
 2.87606052e-03 3.42100020e-04 3.39099331e-05 2.88107110e-06
 2.14185010e-07 1.41537541e-08]
           0         1         2         3         4         5         6  \
0   2.544651  1.513399  0.450037  0.089218  0.013265  0.001578  0.000156   
1   7.828465  4.655879  1.384512  0.274474  0.040810  0.004854  0.000481   
2  12.041899  7.161765  2.129684  0.422201  0.062775  0.007467  0.000740   
3  12.348725  7.344246  2.183948  0.432958  0.064374  0.007657  0.000759   
4   9.497526  5.648532  1.679696  0.332993  0.049511  0.005889  0.000584   
5   5.843713  3.475473  1.033497  0.204886  0.030463  0.003624  0.000359   
6   2.996

In [20]:
def goal_probs(goals):
    for i in range(0,9):
        if i == 0:
            y=goals[i]
            #x =np.array([0,y])
            #x =[[0,y]]
            d = {i:[0,y]}
            #print(x)
        else:
            #z = np.array([y,y+test[i]])
            z = [y, y+goals[i]]
            d1 = {i:[y,y+goals[i]]}
            #x = np.insert(x,z,axis =0)
            #x.append(z)
            d.update(d1)
            y =y + goals[i]
    return d
    
single_line["Prob_Dist_HG"] = single_line["Prob_HG"].apply(lambda x: goal_probs(x))
single_line["Prob_Dist_AG"] = single_line["Prob_AG"].apply(lambda x: goal_probs(x))
print(single_line.iloc[0,]["Prob_Dist_HG"])
single_line.head()


{0: [0, 0.04612318801766841], 1: [0.04612318801766841, 0.18801838372830765], 2: [0.18801838372830765, 0.4062843734735068], 3: [0.4062843734735068, 0.6301117414632434], 4: [0.6301117414632434, 0.8022595783285889], 5: [0.8022595783285889, 0.9081800580396363], 6: [0.9081800580396363, 0.9624897153014258], 7: [0.9624897153014258, 0.9863583399791578], 8: [0.9863583399791578, 0.9955371373356339]}


,league,season,matchdate,fixture,season_gw,Team_Home,Team_Away,AttStr_Home,DefStr_Home,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG,Pred_HG,Pred_AG,Prob_HG,Prob_AG,score,Prob_Dist_HG,Prob_Dist_AG
0,EPL,2019/20,2020-02-24 20:00:00,Liverpool vs West Ham,27,Liverpool,West Ham,1.518461,0.614483,0.773659,1.358462,1.49141,1.251026,3.076439,0.594737,"[0.04612318801766841, 0.14189519571063924, 0.2...","[0.5517075509075227, 0.32812098869639744, 0.09...",0 1 2 3 ...,"{0: [0, 0.04612318801766841], 1: [0.0461231880...","{0: [0, 0.5517075509075227], 1: [0.55170755090..."
34,EPL,2019/20,2020-02-28 20:00:00,Norwich vs Leicester,28,Norwich,Leicester,0.720824,1.243807,1.086597,0.941559,1.49141,1.251026,1.012218,1.690783,"[0.36341208898583166, 0.36785220600806884, 0.1...","[0.18437507602156017, 0.3117382698937874, 0.26...",0 1 2 3 ...,"{0: [0, 0.36341208898583166], 1: [0.3634120889...","{0: [0, 0.18437507602156017], 1: [0.1843750760..."
12,EPL,2019/20,2020-02-29 12:30:00,Brighton vs Crystal Palace,28,Brighton,Crystal Palace,0.864810,1.026824,0.717497,1.026888,1.49141,1.251026,1.324466,0.921684,"[0.26594493689055637, 0.3522350256666123, 0.23...","[0.3978483025925021, 0.36669061210769166, 0.16...",0 1 2 3 ...,"{0: [0, 0.26594493689055637], 1: [0.2659449368...","{0: [0, 0.3978483025925021], 1: [0.39784830259..."
30,EPL,2019/20,2020-02-29 15:00:00,Bournemouth vs Chelsea,28,Bournemouth,Chelsea,0.867874,1.219172,1.242395,0.687826,1.49141,1.251026,0.890292,1.894920,"[0.4105359815561323, 0.3654967773133354, 0.162...","[0.15033040087368443, 0.2848640433441079, 0.26...",0 1 2 3 ...,"{0: [0, 0.4105359815561323], 1: [0.41053598155...","{0: [0, 0.15033040087368443], 1: [0.1503304008..."
43,EPL,2019/20,2020-02-29 15:00:00,West Ham vs Southampton,28,West Ham,Southampton,0.773659,1.358462,0.992140,0.975213,1.49141,1.251026,1.125242,1.686112,"[0.3245740096084095, 0.3652242090783172, 0.205...","[0.18523825915492134, 0.3123325248255356, 0.26...",0 1 2 3 ...,"{0: [0, 0.3245740096084095], 1: [0.32457400960...","{0: [0, 0.18523825915492134], 1: [0.1852382591..."


## Simulating Goals Scored
We will generate random numbers between 0 and 1 for the desired number of simulations. <br>
The number generated equates to a probability and we fill find the corresponding goal for that probability. <br>
We will do this for both home and away goals as they both have different probability distributions. <br>
The first item in the Home Goals simulation list and the first item in the Away Goals simulation list can be considered as the same simulated game. The same goes for the second, and the third etc

In [21]:
#generate random numbers between 0 and 1. each number simulates a game. 

def siumulate_goals(goal_prob):
    goals=[]
    sim = np.random.random_sample(10)
    for i in sim:
        x = max(np.where((pd.DataFrame(goal_prob) <i).max())[0])
        goals.append(x)
    return goals

single_line["HGsims"] = single_line["Prob_Dist_HG"].apply(lambda x: siumulate_goals(x))
single_line["AGsims"] = single_line["Prob_Dist_AG"].apply(lambda x: siumulate_goals(x))
single_line.head()

,league,season,matchdate,fixture,season_gw,Team_Home,Team_Away,AttStr_Home,DefStr_Home,AttStr_Away,...,SeasonAG,Pred_HG,Pred_AG,Prob_HG,Prob_AG,score,Prob_Dist_HG,Prob_Dist_AG,HGsims,AGsims
0,EPL,2019/20,2020-02-24 20:00:00,Liverpool vs West Ham,27,Liverpool,West Ham,1.518461,0.614483,0.773659,...,1.251026,3.076439,0.594737,"[0.04612318801766841, 0.14189519571063924, 0.2...","[0.5517075509075227, 0.32812098869639744, 0.09...",0 1 2 3 ...,"{0: [0, 0.04612318801766841], 1: [0.0461231880...","{0: [0, 0.5517075509075227], 1: [0.55170755090...","[8, 2, 2, 0, 2, 3, 2, 3, 2, 4]","[1, 0, 0, 0, 1, 0, 0, 0, 1, 1]"
34,EPL,2019/20,2020-02-28 20:00:00,Norwich vs Leicester,28,Norwich,Leicester,0.720824,1.243807,1.086597,...,1.251026,1.012218,1.690783,"[0.36341208898583166, 0.36785220600806884, 0.1...","[0.18437507602156017, 0.3117382698937874, 0.26...",0 1 2 3 ...,"{0: [0, 0.36341208898583166], 1: [0.3634120889...","{0: [0, 0.18437507602156017], 1: [0.1843750760...","[2, 0, 1, 0, 0, 0, 0, 2, 1, 2]","[0, 0, 2, 1, 0, 1, 2, 2, 3, 2]"
12,EPL,2019/20,2020-02-29 12:30:00,Brighton vs Crystal Palace,28,Brighton,Crystal Palace,0.864810,1.026824,0.717497,...,1.251026,1.324466,0.921684,"[0.26594493689055637, 0.3522350256666123, 0.23...","[0.3978483025925021, 0.36669061210769166, 0.16...",0 1 2 3 ...,"{0: [0, 0.26594493689055637], 1: [0.2659449368...","{0: [0, 0.3978483025925021], 1: [0.39784830259...","[0, 1, 3, 2, 2, 1, 4, 0, 0, 0]","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]"
30,EPL,2019/20,2020-02-29 15:00:00,Bournemouth vs Chelsea,28,Bournemouth,Chelsea,0.867874,1.219172,1.242395,...,1.251026,0.890292,1.894920,"[0.4105359815561323, 0.3654967773133354, 0.162...","[0.15033040087368443, 0.2848640433441079, 0.26...",0 1 2 3 ...,"{0: [0, 0.4105359815561323], 1: [0.41053598155...","{0: [0, 0.15033040087368443], 1: [0.1503304008...","[0, 2, 1, 1, 1, 1, 0, 1, 0, 1]","[3, 3, 2, 2, 2, 2, 1, 2, 0, 5]"
43,EPL,2019/20,2020-02-29 15:00:00,West Ham vs Southampton,28,West Ham,Southampton,0.773659,1.358462,0.992140,...,1.251026,1.125242,1.686112,"[0.3245740096084095, 0.3652242090783172, 0.205...","[0.18523825915492134, 0.3123325248255356, 0.26...",0 1 2 3 ...,"{0: [0, 0.3245740096084095], 1: [0.32457400960...","{0: [0, 0.18523825915492134], 1: [0.1852382591...","[2, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 2, 3, 1, 1, 1, 0, 2]"


From the simulated Home and Away goals, we will calculate points scored for each team and the goal difference for each team.<br>
We need to calculate both Home Points and Away Points as well as Home Goals and Away Goals so that we can restructure the data to extract performance by team

In [22]:
def points(teamA,teamB):
    gd = np.array(teamA)-np.array(teamB)
    points = np.where(gd>0,3,np.where(gd==0,1,0))
    return points

def goal_dif(teamA,teamB):
    gd = np.array(teamA)-np.array(teamB)
    return gd
    

single_line["homepoints"] = single_line.apply(lambda row: points(row["HGsims"],row["AGsims"]), axis = 1)
single_line["awaypoints"] = single_line.apply(lambda row: points(row["AGsims"],row["HGsims"]), axis = 1)
single_line["HomeGD"] = single_line.apply(lambda row: goal_dif(row["HGsims"],row["AGsims"]), axis = 1)
single_line["AwayGD"] = single_line.apply(lambda row: goal_dif(row["AGsims"],row["HGsims"]), axis = 1)
single_line.head()
        
    

,league,season,matchdate,fixture,season_gw,Team_Home,Team_Away,AttStr_Home,DefStr_Home,AttStr_Away,...,Prob_AG,score,Prob_Dist_HG,Prob_Dist_AG,HGsims,AGsims,homepoints,awaypoints,HomeGD,AwayGD
0,EPL,2019/20,2020-02-24 20:00:00,Liverpool vs West Ham,27,Liverpool,West Ham,1.518461,0.614483,0.773659,...,"[0.5517075509075227, 0.32812098869639744, 0.09...",0 1 2 3 ...,"{0: [0, 0.04612318801766841], 1: [0.0461231880...","{0: [0, 0.5517075509075227], 1: [0.55170755090...","[8, 2, 2, 0, 2, 3, 2, 3, 2, 4]","[1, 0, 0, 0, 1, 0, 0, 0, 1, 1]","[3, 3, 3, 1, 3, 3, 3, 3, 3, 3]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[7, 2, 2, 0, 1, 3, 2, 3, 1, 3]","[-7, -2, -2, 0, -1, -3, -2, -3, -1, -3]"
34,EPL,2019/20,2020-02-28 20:00:00,Norwich vs Leicester,28,Norwich,Leicester,0.720824,1.243807,1.086597,...,"[0.18437507602156017, 0.3117382698937874, 0.26...",0 1 2 3 ...,"{0: [0, 0.36341208898583166], 1: [0.3634120889...","{0: [0, 0.18437507602156017], 1: [0.1843750760...","[2, 0, 1, 0, 0, 0, 0, 2, 1, 2]","[0, 0, 2, 1, 0, 1, 2, 2, 3, 2]","[3, 1, 0, 0, 1, 0, 0, 1, 0, 1]","[0, 1, 3, 3, 1, 3, 3, 1, 3, 1]","[2, 0, -1, -1, 0, -1, -2, 0, -2, 0]","[-2, 0, 1, 1, 0, 1, 2, 0, 2, 0]"
12,EPL,2019/20,2020-02-29 12:30:00,Brighton vs Crystal Palace,28,Brighton,Crystal Palace,0.864810,1.026824,0.717497,...,"[0.3978483025925021, 0.36669061210769166, 0.16...",0 1 2 3 ...,"{0: [0, 0.26594493689055637], 1: [0.2659449368...","{0: [0, 0.3978483025925021], 1: [0.39784830259...","[0, 1, 3, 2, 2, 1, 4, 0, 0, 0]","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]","[1, 1, 3, 3, 3, 3, 3, 1, 1, 1]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1]","[0, 0, 2, 1, 1, 1, 4, 0, 0, 0]","[0, 0, -2, -1, -1, -1, -4, 0, 0, 0]"
30,EPL,2019/20,2020-02-29 15:00:00,Bournemouth vs Chelsea,28,Bournemouth,Chelsea,0.867874,1.219172,1.242395,...,"[0.15033040087368443, 0.2848640433441079, 0.26...",0 1 2 3 ...,"{0: [0, 0.4105359815561323], 1: [0.41053598155...","{0: [0, 0.15033040087368443], 1: [0.1503304008...","[0, 2, 1, 1, 1, 1, 0, 1, 0, 1]","[3, 3, 2, 2, 2, 2, 1, 2, 0, 5]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[3, 3, 3, 3, 3, 3, 3, 3, 1, 3]","[-3, -1, -1, -1, -1, -1, -1, -1, 0, -4]","[3, 1, 1, 1, 1, 1, 1, 1, 0, 4]"
43,EPL,2019/20,2020-02-29 15:00:00,West Ham vs Southampton,28,West Ham,Southampton,0.773659,1.358462,0.992140,...,"[0.18523825915492134, 0.3123325248255356, 0.26...",0 1 2 3 ...,"{0: [0, 0.3245740096084095], 1: [0.32457400960...","{0: [0, 0.18523825915492134], 1: [0.1852382591...","[2, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 2, 3, 1, 1, 1, 0, 2]","[3, 1, 1, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 1, 3, 3, 3, 3, 3, 1, 3]","[1, 0, 0, -2, -3, -1, -1, -1, 0, -2]","[-1, 0, 0, 2, 3, 1, 1, 1, 0, 2]"


In [118]:
home_teams = single_line[["league","season","matchdate","fixture","season_gw","Team_Home","Team_Away",
                          "Pred_HG","Pred_AG","HGsims","AGsims","HomeGD","homepoints"]].copy()
away_teams = single_line[["league","season","matchdate","fixture","season_gw","Team_Home","Team_Away",
                          "Pred_HG","Pred_AG","HGsims","AGsims","AwayGD","awaypoints"]].copy()
home_teams["HomeGame"]=1
away_teams["HomeGame"]=0
home_teams.rename(columns ={"Team_Home":"Team"
                           ,"Team_Away":"Opponent"
                           ,"Pred_HG":"Pred_GF"
                            ,"Pred_AG":"Pred_GA"
                           ,"HGsims":"GoalsFor"
                           ,"AGsims":"GoalsAgainst"
                            ,"HomeGD":"GD"
                           ,"homepoints":"points"},inplace =True)
away_teams.rename(columns ={"Team_Away":"Team"
                           ,"Team_Home":"Opponent"
                           ,"Pred_AG":"Pred_GF"
                            ,"Pred_HG":"Pred_GA"
                           ,"AGsims":"GoalsFor"
                           ,"HGsims":"GoalsAgainst"
                            ,"AwayGD":"GD"
                           ,"awaypoints":"points"},inplace =True)

def results(points,point):
    result =[]
    for i in points:
        if i ==point:
            output = 1
        else:
            output = 0
        result.append(output)
    return result
    
    

col_order = home_teams.columns
away_teams = away_teams[col_order]

team_sims = pd.concat([home_teams,away_teams])
team_sims["Wins"] = team_sims["points"].apply(lambda x: results(x,3))
team_sims["Draws"] = team_sims["points"].apply(lambda x: results(x,1))
team_sims["Losses"] = team_sims["points"].apply(lambda x: results(x,0))

team_sims.sort_values(["Team","matchdate"]).head()
    

,league,season,matchdate,fixture,season_gw,Team,Opponent,Pred_GF,Pred_GA,GoalsFor,GoalsAgainst,GD,points,HomeGame,Wins,Draws,Losses
58,EPL,2019/20,2020-02-29 15:00:00,Manchester City vs Arsenal,28,Arsenal,Manchester City,0.747741,2.659173,"[1, 2, 0, 1, 0, 1, 0, 0, 1, 0]","[3, 2, 2, 4, 2, 2, 6, 2, 1, 1]","[-2, 0, -2, -3, -2, -1, -6, -2, 0, -1]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 0, 1, 1, 1, 1, 1, 1, 0, 1]"
4,EPL,2019/20,2020-03-07 15:00:00,Arsenal vs West Ham,29,Arsenal,West Ham,2.069164,1.000038,"[3, 3, 4, 1, 2, 2, 4, 2, 0, 3]","[1, 1, 4, 1, 0, 1, 0, 3, 4, 3]","[2, 2, 0, 0, 2, 1, 4, -1, -4, 0]","[3, 3, 1, 1, 3, 3, 3, 0, 0, 1]",1,"[1, 1, 0, 0, 1, 1, 1, 0, 0, 0]","[0, 0, 1, 1, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0]"
56,EPL,2019/20,2020-03-14 15:00:00,Brighton vs Arsenal,30,Arsenal,Brighton,1.311935,1.332658,"[3, 2, 2, 1, 3, 1, 0, 2, 1, 1]","[0, 1, 0, 2, 2, 2, 0, 2, 1, 1]","[3, 1, 2, -1, 1, -1, 0, 0, 0, 0]","[3, 3, 3, 0, 3, 0, 1, 1, 1, 1]",0,"[1, 1, 1, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
60,EPL,2019/20,2020-03-21 15:00:00,Southampton vs Arsenal,31,Arsenal,Southampton,1.245994,1.528871,"[2, 2, 3, 0, 0, 3, 2, 1, 2, 2]","[1, 2, 1, 0, 2, 2, 0, 0, 1, 4]","[1, 0, 2, 0, -2, 1, 2, 1, 1, -2]","[3, 1, 3, 1, 0, 3, 3, 3, 3, 0]",0,"[1, 0, 1, 0, 0, 1, 1, 1, 1, 0]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]"
103,EPL,2019/20,2020-04-04 14:00:00,Arsenal vs Norwich,32,Arsenal,Norwich,1.894526,0.931744,"[0, 1, 1, 1, 3, 1, 1, 2, 3, 2]","[1, 1, 0, 0, 1, 1, 1, 1, 1, 0]","[-1, 0, 1, 1, 2, 0, 0, 1, 2, 2]","[0, 1, 3, 3, 3, 1, 1, 3, 3, 3]",1,"[0, 0, 1, 1, 1, 0, 0, 1, 1, 1]","[0, 1, 0, 0, 0, 1, 1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


Here we're experimenting with extracting all of the simulations for one team and attempting to sum them up in the correct order. An extra set of brackets around the list of arrays seems to be key for the function to be processed in the correct order.

In [58]:
#looking at the simulations for one team
test = list(team_sims[team_sims["Team"]=="Liverpool"]["points"])
print([test][0])
tl =[]
print()
for x in [test]:
    tl.append(sum(x))
print(tl)
test

[array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 1, 3, 3, 3, 3, 3]), array([1, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([0, 3, 3, 3, 3, 3, 3, 0, 3, 3]), array([1, 3, 0, 0, 0, 3, 0, 1, 3, 3]), array([3, 0, 3, 3, 3, 1, 1, 1, 3, 3]), array([0, 0, 3, 1, 1, 3, 0, 1, 3, 0]), array([0, 1, 0, 3, 0, 0, 3, 1, 3, 0]), array([1, 3, 3, 3, 3, 1, 3, 3, 3, 1]), array([3, 3, 3, 1, 3, 1, 3, 3, 0, 3]), array([3, 3, 3, 3, 3, 3, 0, 3, 3, 3])]

[array([21, 28, 30, 27, 26, 27, 25, 25, 33, 28])]


[array([3, 3, 3, 1, 3, 3, 3, 3, 3, 3]),
 array([3, 3, 3, 3, 1, 3, 3, 3, 3, 3]),
 array([1, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([0, 3, 3, 3, 3, 3, 3, 0, 3, 3]),
 array([1, 3, 0, 0, 0, 3, 0, 1, 3, 3]),
 array([3, 0, 3, 3, 3, 1, 1, 1, 3, 3]),
 array([0, 0, 3, 1, 1, 3, 0, 1, 3, 0]),
 array([0, 1, 0, 3, 0, 0, 3, 1, 3, 0]),
 array([1, 3, 3, 3, 3, 1, 3, 3, 3, 1]),
 array([3, 3, 3, 1, 3, 1, 3, 3, 0, 3]),
 array([3, 3, 3, 3, 3, 3, 0, 3, 3, 3])]

In [50]:
for x in [test]:
    print(sum(x))

[21 28 30 27 26 27 25 25 33 28]


There is a list function available as part of the group by and aggregate functions for pandas. This is useful for what we're tryin to do for each team. Here's another one off check to ensure we're following the same simulation all the way through.

In [35]:
liv = team_sims[team_sims["Team"]=="Liverpool"].groupby("Team").agg({"points":list}).reset_index()
print(liv)
print(liv["points"])
# zl = zip(*liv)
print([sum(x) for x in liv["points"]])
# print([sum(x) for x in liv])

        Team                                             points
0  Liverpool  [[3, 3, 3, 1, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...
0    [[3, 3, 3, 1, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...
Name: points, dtype: object
[array([21, 28, 30, 27, 26, 27, 25, 25, 33, 28])]


In [51]:
#run team_sum first in code below
t2 = team_sum[team_sum["Team"]=="Liverpool"]["points"].copy()
print(type(t2))
[sum(x) for x in t2]

<class 'pandas.core.series.Series'>


[array([21, 28, 30, 27, 26, 27, 25, 25, 33, 28])]

### Applying it across all teams

In [119]:
team_sum = team_sims.groupby("Team").agg({"GD":list
                                     ,"points":list}).reset_index()
team_sum

,Team,GD,points
0,Arsenal,"[[2, 2, 0, 0, 2, 1, 4, -1, -4, 0], [-1, 0, 1, ...","[[3, 3, 1, 1, 3, 3, 3, 0, 0, 1], [0, 1, 3, 3, ..."
1,Aston Villa,"[[0, 0, -4, -2, -1, -3, -1, 1, 2, 1], [0, -2, ...","[[1, 1, 0, 0, 0, 0, 0, 3, 3, 3], [1, 0, 1, 0, ..."
2,Bournemouth,"[[-3, -1, -1, -1, -1, -1, -1, -1, 0, -4], [-1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 3, 3, 3, ..."
3,Brighton,"[[0, 0, 2, 1, 1, 1, 4, 0, 0, 0], [-3, -1, -2, ...","[[1, 1, 3, 3, 3, 3, 3, 1, 1, 1], [0, 0, 0, 3, ..."
4,Burnley,"[[2, 1, -1, -3, -2, 1, 0, -1, 2, -1], [-1, 3, ...","[[3, 3, 0, 0, 0, 3, 1, 0, 3, 0], [0, 3, 3, 1, ..."
5,Chelsea,"[[1, 1, 0, 0, 1, 4, -2, 2, 0, 0], [-1, 0, -1, ...","[[3, 3, 1, 1, 3, 3, 0, 3, 1, 1], [0, 1, 0, 0, ..."
6,Crystal Palace,"[[3, 1, -1, 0, 0, 1, 1, 0, -1, 1], [0, 1, -1, ...","[[3, 3, 0, 1, 1, 3, 3, 1, 0, 3], [1, 3, 0, 3, ..."
7,Everton,"[[-1, 2, -2, -2, 2, 1, -1, 0, 1, 1], [1, 1, -2...","[[0, 3, 0, 0, 3, 3, 0, 1, 3, 3], [3, 3, 0, 1, ..."
8,Leicester,"[[0, 4, 3, 1, 3, 2, 3, 3, 0, 0], [-1, -4, 2, 0...","[[1, 3, 3, 3, 3, 3, 3, 3, 1, 1], [0, 0, 3, 1, ..."
9,Liverpool,"[[7, 2, 2, 0, 1, 3, 2, 3, 1, 3], [4, 3, 2, 4, ...","[[3, 3, 3, 1, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ..."


In [120]:
def sum_array(var):
    sums = [sum(x) for x in var]
    return sums[0]

team_sum["GD"] = team_sum.apply(lambda row: sum_array(row[["GD"]]),axis = 1) #double brackets here or double brackets in the function
team_sum["points"] = team_sum.apply(lambda row: sum_array(row[["points"]]),axis=1)


team_sum.head(20)

,Team,GD,points
0,Arsenal,"[0, 10, 6, -2, 7, 1, -2, -5, 0, 6]","[16, 24, 22, 14, 22, 14, 13, 14, 17, 16]"
1,Aston Villa,"[-13, -19, -17, -15, -23, -14, -14, -7, -11, -18]","[6, 5, 9, 5, 3, 4, 5, 13, 8, 9]"
2,Bournemouth,"[-19, -1, 0, -13, -3, -10, -14, -18, -8, -12]","[8, 17, 13, 6, 10, 9, 2, 3, 7, 13]"
3,Brighton,"[-3, -5, -7, 5, 2, 1, -8, 5, -6, -3]","[17, 9, 10, 17, 16, 21, 6, 18, 10, 14]"
4,Burnley,"[-6, 5, -4, -8, -6, -2, 4, -1, 3, -6]","[11, 18, 11, 10, 11, 13, 21, 11, 16, 12]"
5,Chelsea,"[9, 8, 3, 15, 16, 13, 16, 8, 5, 10]","[17, 21, 18, 26, 27, 21, 27, 23, 21, 23]"
6,Crystal Palace,"[3, -7, -15, -3, -10, 1, -15, 1, -10, -2]","[16, 10, 2, 12, 7, 15, 7, 18, 8, 14]"
7,Everton,"[3, 11, -9, -3, 6, -7, -2, 8, -1, 6]","[15, 20, 10, 14, 16, 15, 13, 18, 15, 20]"
8,Leicester,"[9, -5, 18, 1, 1, 6, 10, 7, 1, 0]","[19, 9, 26, 17, 17, 15, 24, 19, 17, 13]"
9,Liverpool,"[18, 13, 27, 19, 15, 20, 10, 16, 24, 24]","[21, 28, 30, 27, 26, 27, 25, 25, 33, 28]"


In [61]:
#qa on our liverpool dataset earlier
liv.apply(lambda row: sum_array(row[["points"]]),axis =1)


0    [21, 28, 30, 27, 26, 27, 25, 25, 33, 28]
dtype: object

### Ranking the teams
Now that we've simulated the number of points for each season. We now need to rank each points total for each team.

In [116]:
## zipping does a cool thing that I don't fully understand
zipped = list(zip(*team_sum["points"]))
#rk = [pd.Series(i).rank(method = "min",ascending = False) for i in zipped]
print(zipped)
print()

ranks = []
for i in zipped:
    print(i)
    rk = pd.Series(i).rank(method = "min", ascending = False)
    print(rk[0])
    #print(pd.Series(i).rank(method = "min",ascending = False))# can print this as an array for ease of visualising
    ranks.append(rk[0])
ranks

[(16, 6, 8, 17, 11, 17, 16, 15, 19, 21, 27, 19, 12, 17, 22, 6, 15, 9, 10, 20), (24, 5, 17, 9, 18, 21, 10, 20, 9, 28, 20, 19, 6, 15, 13, 14, 12, 10, 8, 26), (22, 9, 13, 10, 11, 18, 2, 10, 26, 30, 24, 19, 10, 6, 18, 14, 15, 14, 13, 24), (14, 5, 6, 17, 10, 26, 12, 14, 17, 27, 26, 20, 11, 6, 20, 11, 12, 11, 11, 20), (22, 3, 10, 16, 11, 27, 7, 16, 17, 26, 28, 24, 13, 7, 13, 16, 16, 14, 7, 17), (14, 4, 9, 21, 13, 21, 15, 15, 15, 27, 27, 21, 8, 9, 16, 17, 18, 12, 11, 11), (13, 5, 2, 6, 21, 27, 7, 13, 24, 25, 26, 22, 23, 14, 17, 17, 23, 2, 6, 21), (14, 13, 3, 18, 11, 23, 18, 18, 19, 25, 29, 18, 7, 8, 15, 13, 15, 9, 14, 20), (17, 8, 7, 10, 16, 21, 8, 15, 17, 33, 23, 18, 13, 7, 11, 14, 26, 18, 10, 16), (16, 9, 13, 14, 12, 23, 14, 20, 13, 28, 23, 12, 4, 2, 5, 31, 17, 10, 16, 22)]

(16, 6, 8, 17, 11, 17, 16, 15, 19, 21, 27, 19, 12, 17, 22, 6, 15, 9, 10, 20)
10.0
(24, 5, 17, 9, 18, 21, 10, 20, 9, 28, 20, 19, 6, 15, 13, 14, 12, 10, 8, 26)
3.0
(22, 9, 13, 10, 11, 18, 2, 10, 26, 30, 24, 19, 10, 6, 18,

[10.0, 3.0, 5.0, 9.0, 5.0, 12.0, 13.0, 12.0, 7.0, 8.0]

## Ranking all teams
We can use the index of our table as a variable in our function to keep the item of the list that we're interested in for each team.

In [121]:
#run this only once to avoid creating unnecessary columns
team_sum.reset_index(inplace =True)
team_sum.head(1)

,index,Team,GD,points
0,0,Arsenal,"[0, 10, 6, -2, 7, 1, -2, -5, 0, 6]","[16, 24, 22, 14, 22, 14, 13, 14, 17, 16]"


In [122]:
def rank_array(var):
    ranks = []
    for i in zipped:
        #print(i)
        rk = pd.Series(i).rank(method = "min", ascending = False)
        #print(rk[0])
        #print(pd.Series(i).rank(method = "min",ascending = False))# can print this as an array for ease of visualising
        ranks.append(rk[var])
    return ranks


team_sum["ranks"]= team_sum["index"].apply(lambda x: rank_array(x))
#team_sum["new_ranks"] = team_sum["ranks"].apply(lambda row: rank_keep(row[["ranks"]],row["index"]),axis = 1)


team_sum

,index,Team,GD,points,ranks
0,0,Arsenal,"[0, 10, 6, -2, 7, 1, -2, -5, 0, 6]","[16, 24, 22, 14, 22, 14, 13, 14, 17, 16]","[10.0, 3.0, 5.0, 9.0, 5.0, 12.0, 13.0, 12.0, 7..."
1,1,Aston Villa,"[-13, -19, -17, -15, -23, -14, -14, -7, -11, -18]","[6, 5, 9, 5, 3, 4, 5, 13, 8, 9]","[19.0, 20.0, 18.0, 20.0, 20.0, 20.0, 18.0, 14...."
2,2,Bournemouth,"[-19, -1, 0, -13, -3, -10, -14, -18, -8, -12]","[8, 17, 13, 6, 10, 9, 2, 3, 7, 13]","[18.0, 9.0, 12.0, 18.0, 16.0, 17.0, 19.0, 20.0..."
3,3,Brighton,"[-3, -5, -7, 5, 2, 1, -8, 5, -6, -3]","[17, 9, 10, 17, 16, 21, 6, 18, 10, 14]","[7.0, 16.0, 15.0, 7.0, 8.0, 3.0, 16.0, 6.0, 15..."
4,4,Burnley,"[-6, 5, -4, -8, -6, -2, 4, -1, 3, -6]","[11, 18, 11, 10, 11, 13, 21, 11, 16, 12]","[15.0, 8.0, 14.0, 17.0, 15.0, 13.0, 8.0, 16.0,..."
5,5,Chelsea,"[9, 8, 3, 15, 16, 13, 16, 8, 5, 10]","[17, 21, 18, 26, 27, 21, 27, 23, 21, 23]","[7.0, 4.0, 7.0, 2.0, 2.0, 3.0, 1.0, 3.0, 4.0, ..."
6,6,Crystal Palace,"[3, -7, -15, -3, -10, 1, -15, 1, -10, -2]","[16, 10, 2, 12, 7, 15, 7, 18, 8, 14]","[10.0, 14.0, 20.0, 11.0, 17.0, 9.0, 15.0, 6.0,..."
7,7,Everton,"[3, 11, -9, -3, 6, -7, -2, 8, -1, 6]","[15, 20, 10, 14, 16, 15, 13, 18, 15, 20]","[12.0, 5.0, 15.0, 9.0, 8.0, 9.0, 13.0, 6.0, 11..."
8,8,Leicester,"[9, -5, 18, 1, 1, 6, 10, 7, 1, 0]","[19, 9, 26, 17, 17, 15, 24, 19, 17, 13]","[5.0, 16.0, 2.0, 7.0, 6.0, 9.0, 4.0, 5.0, 7.0,..."
9,9,Liverpool,"[18, 13, 27, 19, 15, 20, 10, 16, 24, 24]","[21, 28, 30, 27, 26, 27, 25, 25, 33, 28]","[3.0, 1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 2.0, 1.0, ..."


In [133]:
team_sum.ranks.apply(np.mean)

0      8.4
1     18.3
2     16.0
3     10.3
4     12.9
5      3.6
6     12.9
7      9.4
8      7.3
9      1.8
10     2.2
11     6.1
12    14.9
13    15.7
14     9.8
15    10.5
16     8.3
17    13.7
18    14.2
19     6.0
Name: ranks, dtype: float64

In [67]:
t4 = team_sum[team_sum["Team"].isin(["Liverpool"])]
def rank_keep(var,index):
    output = []
    ranks = list(zip(*var))
    print(ranks)
    for i in ranks:
        keep = ranks[index][index]
        output.append(keep)
    return output
t4.loc[:,"test"] = t4.apply(lambda row: rank_keep(row["ranks"],row["index"]),axis = 1)
t4.head()

KeyError: ('index', 'occurred at index 9')

In [559]:
t3 = team_sum[team_sum["Team"]=="Arsenal"][["ranks"]]
t3
tz = list(zip(*t3))
tz
# for i in tz:
#     print(i[0][0])

[('r',), ('a',), ('n',), ('k',), ('s',)]

In [78]:
type(team_sum["points"])

pandas.core.series.Series

In [375]:
def sum_array(var):
    sums = [sum(x) for x in var]
    return sums
    
test["GD"].apply(lambda x: sum_array(x))

0           [8, 8, 1, -13, 1, -13, -5, -4, -11, -1, 17]
1     [-4, -23, -21, -18, -1, -17, -23, 9, -28, -18, 2]
2          [-11, 7, 9, 7, -4, -3, -13, -3, -7, -33, -4]
3           [9, 5, -4, -22, -24, 4, -7, -6, 11, -7, -6]
4            [5, 9, -15, -5, 6, 3, -17, 5, 13, -14, -1]
5             [1, -4, 9, 32, 13, 11, 23, 10, 12, 0, -4]
6        [2, -5, -12, -13, -3, -9, -7, -26, -13, 1, -4]
7             [-10, -5, 1, 4, 18, 4, -1, 16, 4, -13, 7]
8              [23, 6, 13, 1, -9, 12, 1, -1, -1, 4, 10]
9       [17, 13, 26, 28, 14, 4, 24, 5, -12, 22, 13, 26]
10          [13, 17, 12, 26, 33, 25, 13, 4, 10, 24, 12]
11            [-13, -5, 7, 6, 13, 10, 11, 4, 18, 13, 9]
12     [-3, -5, -9, -1, -9, -26, -11, -9, -26, -13, -4]
13     [-12, 2, -16, -11, 1, 1, -11, -8, -11, -32, -25]
14               [11, 7, 3, 0, -7, 4, 5, 5, 15, -1, -6]
15            [11, 4, -10, 7, 6, -3, -2, -9, -4, -6, 3]
16          [-4, -11, 13, -4, 1, -10, -5, -7, -7, 9, 3]
17         [-24, -1, 9, 11, 13, -12, -2, -9, -9,

In [373]:
su = [sum(x) for x in t1]
su

[8, 8, 1, -13, 1, -13, -5, -4, -11, -1, 17]

In [287]:
#for i in range(0,10):
sum_df = team_sims.copy()
sum_df["g1"] = sum_df.GoalsFor.apply(lambda x: x[0])
sum_df.groupby("Team")["g1"].sum()
#team_sims["c1"] = 
#team_sims[["league","season","Team","c1"]]

Team
Arsenal                    18
Aston Villa                 6
Bournemouth                19
Brighton                   15
Burnley                    18
Chelsea                    19
Crystal Palace              9
Everton                    17
Leicester                  25
Liverpool                  33
Manchester City            24
Manchester United          12
Newcastle United           10
Norwich                    10
Sheffield United           31
Southampton                22
Tottenham                  18
Watford                    25
West Ham                   19
Wolverhampton Wanderers    19
Name: g1, dtype: int64

In [141]:
np.random.random_sample(10)

array([0.65980057, 0.1239936 , 0.50402719, 0.07577183, 0.81432896,
       0.59045463, 0.7420434 , 0.46777356, 0.4394528 , 0.14061111])

In [178]:
d = single_line.iloc[0,]["Prob_Dist_HG"]
print(d)
goals=[]
vals=[]
sim = np.random.random_sample(10)
for i in sim:
    x = max(np.where((pd.DataFrame(d) <i).max())[0])
    goals.append(x)
    vals.append(i)
print(goals)
print(vals)
np.mean(goals)

{0: [0, 0.04612318801766841], 1: [0.04612318801766841, 0.18801838372830765], 2: [0.18801838372830765, 0.4062843734735068], 3: [0.4062843734735068, 0.6301117414632434], 4: [0.6301117414632434, 0.8022595783285889], 5: [0.8022595783285889, 0.9081800580396363], 6: [0.9081800580396363, 0.9624897153014258], 7: [0.9624897153014258, 0.9863583399791578], 8: [0.9863583399791578, 0.9955371373356339]}
[1, 1, 2, 1, 5, 4, 4, 1, 5, 0]
[0.16648379614864095, 0.06786610753199973, 0.39414268327843394, 0.17628341929129576, 0.8633704650512778, 0.753876382674834, 0.6502778200616631, 0.08278814028822123, 0.8090304223241265, 0.026235760035818734]


2.4

In [182]:
ag = goals.copy()
print(hg)
print(ag)
np.array(hg)-np.array(ag)

[1, 0, 1, 2, 4, 3, 6, 2, 5, 1]
[1, 1, 2, 1, 5, 4, 4, 1, 5, 0]


array([ 0, -1, -1,  1, -1, -1,  2,  1,  0,  1], dtype=int64)

In [ ]:
season_av_df = season_average(results_df)

In [22]:
team_df[team_df["Team"]=="Liverpool"][["league","season","matchdate","Team","fixture","season_gw","GoalsFor","GoalsAgainst","xG","xGA"
        ,"xG_av","xGA_av","xG_av_gw","xGA_av_gw"]].tail(38)

,league,season,matchdate,Team,fixture,season_gw,GoalsFor,GoalsAgainst,xG,xGA,xG_av,xGA_av,xG_av_gw,xGA_av_gw
7814,EPL,2019/20,2019-08-09 20:00:00,Liverpool,Liverpool vs Norwich,1,4.0,1.0,2.23456,0.842407,2.059146,0.780836,1.429207,1.382166
7815,EPL,2019/20,2019-08-17 15:00:00,Liverpool,Southampton vs Liverpool,2,2.0,1.0,1.52022,2.140470,2.068628,0.784164,1.427151,1.382859
7816,EPL,2019/20,2019-08-24 17:30:00,Liverpool,Liverpool vs Arsenal,3,3.0,1.0,2.78821,0.985542,2.038984,0.857479,1.438432,1.354007
7817,EPL,2019/20,2019-08-31 17:30:00,Liverpool,Burnley vs Liverpool,4,3.0,0.0,1.20057,0.617390,2.079484,0.864402,1.417125,1.358491
7818,EPL,2019/20,2019-09-14 11:30:00,Liverpool,Liverpool vs Newcastle United,5,3.0,1.0,2.93281,0.291073,2.031974,0.851050,1.417555,1.384616
7819,EPL,2019/20,2019-09-22 15:30:00,Liverpool,Chelsea vs Liverpool,6,2.0,1.0,1.03003,1.233210,2.080668,0.820780,1.425020,1.391144
7820,EPL,2019/20,2019-09-28 11:30:00,Liverpool,Sheffield United vs Liverpool,7,1.0,0.0,1.97079,1.170220,2.023876,0.843074,1.422208,1.393102
7821,EPL,2019/20,2019-10-05 15:00:00,Liverpool,Liverpool vs Leicester,8,2.0,1.0,3.74867,0.098211,2.021007,0.860758,1.428604,1.402407
7822,EPL,2019/20,2019-10-20 16:30:00,Liverpool,Manchester United vs Liverpool,9,1.0,1.0,1.03212,0.791862,2.114395,0.819538,1.422394,1.406603
7823,EPL,2019/20,2019-10-27 16:30:00,Liverpool,Liverpool vs Tottenham,10,2.0,1.0,2.18851,1.351430,2.055893,0.818042,1.409927,1.391430


In [49]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 3736 to 4115
Data columns (total 16 columns):
league                380 non-null object
season                380 non-null object
matchid               380 non-null int64
matchdate             380 non-null object
fixture               380 non-null object
hometeam              380 non-null object
awayteam              380 non-null object
homegoals             269 non-null float64
awaygoals             269 non-null float64
hxG                   269 non-null float64
axG                   269 non-null float64
season_id             380 non-null int64
league_teams_count    380 non-null int64
season_gw             380 non-null int64
HomePoints            380 non-null int64
AwayPoints            380 non-null int64
dtypes: float64(4), int64(6), object(6)
memory usage: 50.5+ KB


In [43]:


season_av_df = results_df.groupby(['league','season','season_gw']).agg({"homegoals":"sum"
                                                                       ,"awaygoals":"sum"
                                                                       ,"fixture":"nunique"}).reset_index()

season_totals = (season_av_df.groupby(['league','season'])["homegoals","awaygoals","fixture"]
                 .transform(lambda row: row.rolling(window=36,min_periods=2).sum()).shift(1))
season_totals

,homegoals,awaygoals,fixture
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,29.0,23.0,20.0
3,41.0,41.0,30.0
4,60.0,53.0,40.0
5,79.0,66.0,50.0
6,99.0,77.0,60.0
7,118.0,87.0,70.0
8,129.0,101.0,80.0
9,140.0,108.0,90.0


In [40]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10956 entries, 0 to 10955
Data columns (total 16 columns):
league                10956 non-null object
season                10956 non-null object
matchid               10956 non-null int64
matchdate             10956 non-null object
fixture               10956 non-null object
hometeam              10956 non-null object
awayteam              10956 non-null object
homegoals             10359 non-null float64
awaygoals             10359 non-null float64
hxG                   10359 non-null float64
axG                   10359 non-null float64
season_id             10956 non-null int64
league_teams_count    10956 non-null int64
season_gw             10956 non-null int64
HomePoints            10956 non-null int64
AwayPoints            10956 non-null int64
dtypes: float64(4), int64(6), object(6)
memory usage: 1.3+ MB
